In [ ]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append("/mnt/raid/C1_ML_Analysis/source/us-famli-pl/src/")
import loaders.ultrasound_dataset as usd

In [ ]:
def efw_ds_creation():
    date_split_b2a = '2025-11-12'
    date_split_labor = '2025-05-27'

    # Data loading code
    train_b2a_df = pd.read_csv(f'/mnt/famli_netapp_shared/C1_ML_Analysis/famli_ml_lists/dataset_list/{date_split_b2a}/b2a_instance_EFW_train_with_bw_dataset.csv')
    train_b2a_df = train_b2a_df.loc[(train_b2a_df['delivery_ega'] - train_b2a_df['ega']) <= 7.0,:].reset_index(drop=True)
    train_b2a_df.file_path = train_b2a_df.file_path.str.replace('.*Dataset_B2/','Dataset_B2_masked_resampled_256_spc075/',regex=True)
    train_b2a_df.file_path = train_b2a_df.file_path.str.replace('dcm','nrrd', regex=False)
    train_b2a_df['efw_gt'] = train_b2a_df['birthweight_g']
    train_b2a_df['study_id'] = train_b2a_df['visitid']
    train_b2a_df['dataset'] = 'b2a'
    train_b2a_df['tag'] = train_b2a_df['tag'].replace({'IM': 'M', 'IC1': 'C1', 'IC2': 'C2', 'IL0': 'L0', 'IR0': 'R0'})
    train_b2a_df = train_b2a_df.loc[:,["study_id", "file_path", "efw_gt",'tag', 'dataset']].reset_index(drop=True)
    train_labor_df = pd.read_csv(f'/mnt/famli_netapp_shared/C1_ML_Analysis/famli_ml_lists/dataset_list/{date_split_labor}/LABOR_EFW_train_with_bw_dataset.csv')
    train_labor_df = train_labor_df.loc[train_labor_df.site == 4, :].reset_index(drop=True) #Include only Zambia
    train_labor_df['efw_gt'] = train_labor_df.bw
    train_labor_df['study_id'] = train_labor_df.pid
    train_labor_df['dataset'] = 'labor'
    train_labor_df.file_path = train_labor_df.file_path.str.replace('dcm','nrrd', regex=False)
    train_labor_df.file_path = train_labor_df.file_path.str.replace('.*Dataset_A3/','Dataset_A3_masked_resampled_256_spc075/', regex=True)
    train_labor_df = train_labor_df.loc[:,["study_id", "file_path", "efw_gt",'tag', 'dataset']].reset_index(drop=True)
    train_df = pd.concat([train_labor_df, train_b2a_df], axis=0).reset_index(drop=True)
    train_df['file_path'].to_csv(f'/mnt/raid/C1_ML_Analysis/CSV_files/efw_{date_split_b2a}_train_file_paths.csv', index=False)

    train_df = train_df[train_df['file_path'].apply(lambda x: os.path. exists(os.path.join('/mnt/raid/C1_ML_Analysis/', x)))].reset_index(drop=True)
    train_df.to_csv(f'/mnt/raid/C1_ML_Analysis/CSV_files/efw_{date_split_b2a}_train.csv', index=False)
    print(np.unique(train_df.tag, return_counts = True))

    tune_b2a_df = pd.read_csv(f'/mnt/famli_netapp_shared/C1_ML_Analysis/famli_ml_lists/dataset_list/{date_split_b2a}/b2a_instance_EFW_tune_with_bw_dataset.csv')
    tune_b2a_df = tune_b2a_df.loc[(tune_b2a_df['delivery_ega'] - tune_b2a_df['ega']) <= 7.0,:].reset_index(drop=True)
    tune_b2a_df.file_path = tune_b2a_df.file_path.str.replace('.*Dataset_B2/','Dataset_B2_masked_resampled_256_spc075/',regex=True)
    tune_b2a_df.file_path = tune_b2a_df.file_path.str.replace('dcm','nrrd', regex=False)
    tune_b2a_df['efw_gt'] = tune_b2a_df['birthweight_g']
    tune_b2a_df['study_id'] = tune_b2a_df['visitid']
    tune_b2a_df['dataset'] = 'b2a'
    tune_b2a_df = tune_b2a_df.loc[:,["study_id", "file_path", "efw_gt",'tag', 'dataset']].reset_index(drop=True)
    tune_labor_df = pd.read_csv(f'/mnt/famli_netapp_shared/C1_ML_Analysis/famli_ml_lists/dataset_list/{date_split_labor}/LABOR_EFW_tune_with_bw_dataset.csv')
    tune_labor_df = tune_labor_df.loc[tune_labor_df.site == 4, :].reset_index(drop=True) #Include only Zambia
    tune_labor_df['efw_gt'] = tune_labor_df.bw
    tune_labor_df['study_id'] = tune_labor_df.pid
    tune_labor_df['dataset'] = 'labor'
    tune_labor_df.file_path = tune_labor_df.file_path.str.replace('dcm','nrrd', regex=False)
    tune_labor_df.file_path = tune_labor_df.file_path.str.replace('.*Dataset_A3/','Dataset_A3_masked_resampled_256_spc075/', regex=True)
    tune_labor_df = tune_labor_df.loc[:,["study_id", "file_path", "efw_gt",'tag', 'dataset']].reset_index(drop=True)
    val_df = pd.concat([tune_b2a_df, tune_labor_df], axis=0).reset_index(drop=True)
    val_df['file_path'].to_csv(f'/mnt/raid/C1_ML_Analysis/CSV_files/efw_{date_split_b2a}_val_file_paths.csv', index=False)

    val_df = val_df[val_df['file_path'].apply(lambda x: os.path. exists(os.path.join('/mnt/raid/C1_ML_Analysis/', x)))].reset_index(drop=True)
    val_df.to_csv(f'/mnt/raid/C1_ML_Analysis/CSV_files/efw_{date_split_b2a}_val.csv', index=False)
    print(np.unique(val_df.tag, return_counts = True))

    
    

# efw_ds_creation()

In [ ]:



def efw_test_ds():

    date_split_b2a = '2025-11-12'
    date_split_labor = '2025-05-27'
    
    test_b2a_efw = pd.read_csv(f'/mnt/famli_netapp_shared/C1_ML_Analysis/famli_ml_lists/dataset_list/{date_split_b2a}/b2a_instance_EFW_test_with_bw_dataset.csv')
    test_b2a_efw.file_path = test_b2a_efw.file_path.str.replace('.*Dataset_B2/','Dataset_B2_masked_resampled_256_spc075/',regex=True)
    test_b2a_efw.file_path = test_b2a_efw.file_path.str.replace('dcm','nrrd', regex=False)
    test_b2a_efw['efw_gt'] = test_b2a_efw['birthweight_g']
    test_b2a_efw['study_id'] = test_b2a_efw['visitid']
    test_b2a_efw['dataset'] = 'b2a'
    test_b2a_efw = test_b2a_efw.loc[:,["study_id", "file_path", "efw_gt", 'ega', 'tag', 'dataset']].reset_index(drop=True)

    test_labor_df = pd.read_csv(f'/mnt/famli_netapp_shared/C1_ML_Analysis/famli_ml_lists/dataset_list/{date_split_labor}/LABOR_EFW_test_with_bw_dataset.csv')
    test_labor_df['efw_gt'] = test_labor_df.bw
    test_labor_df['study_id'] = test_labor_df.pid
    test_labor_df.file_path = test_labor_df.file_path.str.replace('dcm','nrrd', regex=False)
    test_labor_df.file_path = test_labor_df.file_path.str.replace('.*Dataset_A3/','Dataset_A3_masked_resampled_256_spc075/', regex=True)
    test_labor_df['dataset'] = 'LABOR'
    test_labor_df['ega'] = test_labor_df['best_ega']
    test_labor_df = test_labor_df.loc[:,["study_id", "file_path", "efw_gt",'ega', 'tag', 'dataset']].reset_index(drop=True)
    test_df = pd.concat([test_b2a_efw, test_labor_df], axis=0).reset_index(drop=True)

    test_df['file_path'].to_csv(f'/mnt/raid/C1_ML_Analysis/CSV_files/efw_{date_split_b2a}_test_file_paths.csv', index=False)

    test_df = test_df[test_df['file_path'].apply(lambda x: os.path. exists(os.path.join('/mnt/raid/C1_ML_Analysis/', x)))].reset_index(drop=True)
    test_df.to_csv(f'/mnt/raid/C1_ML_Analysis/CSV_files/efw_{date_split_b2a}_test.csv', index=False)

# efw_test_ds()


In [ ]:
def efw_test_dxa_ds():

    test_df = pd.read_csv('/mnt/famli_netapp_shared/C1_ML_Analysis/famli_ml_lists/dataset_list/2025-11-04/dxa_instance_EFW_with_bw_dataset.csv')
    test_df_ac = pd.read_csv('/mnt/famli_netapp_shared/C1_ML_Analysis/famli_ml_lists/dataset_list/2025-11-04/dxa_instance_EFW_AC_with_bw_dataset.csv')
    test_df_ac['session_id'] = test_df_ac['session_id'] + '-flyto-ac'
    test_df = pd.concat([test_df, test_df_ac], axis=0).reset_index(drop=True)
    # test_df = test_df.loc[test_df.manufacturer == 'Butterfly Network Inc',:].reset_index(drop=True)
    print('Number of visits: ', test_df.visitid.nunique())
    # test_df = test_df[test_df["session_id"].str.contains("ga-tool-novice", na=False)]
    test_df.file_path = test_df.file_path.str.replace('.*DXA/','DXA_masked_resampled_256_spc075/', regex=True)
    test_df.file_path = test_df.file_path.str.replace('dcm','nrrd', regex=False)
    test_df.file_path = test_df.file_path.str.replace('mp4','nrrd', regex=False)
    test_df['efw_gt'] = test_df['birthweight_g']
    test_df['study_id'] = test_df['session_id']
    test_df['dataset'] = 'dxa'
    sweep_counts = test_df.groupby("session_id").size().reset_index(name="count")
    print('Session with less than 8 sweeps:')
    print(sweep_counts[sweep_counts["count"] < 8])
    test_df = test_df.loc[:,["study_id", "file_path", "efw_gt",'tag', 'dataset']].reset_index(drop=True)

    # test_df['file_path'].to_csv('/mnt/raid/C1_ML_Analysis/CSV_files/efw_2025-11-04_dxa_test_file_paths.csv', index=False)

    test_df = test_df[test_df['file_path'].apply(lambda x: os.path.exists(os.path.join('/mnt/raid/C1_ML_Analysis/', x)))].reset_index(drop=True)
    test_df.to_csv('/mnt/raid/C1_ML_Analysis/CSV_files/efw_2025-11-04_dxa_test.csv', index=False)



# efw_test_dxa_ds()

In [ ]:
df_train = pd.read_csv('/mnt/raid/C1_ML_Analysis/CSV_files/efw_2025-10-31_train.csv')

In [ ]:
df_train['efw_gt'].hist(bins=50)

In [ ]:
(df_train['efw_gt']/1000).describe()

In [ ]:

args_dict = {
    "mount_point": "/mnt/raid/C1_ML_Analysis/",
    "csv_train": "/mnt/raid/C1_ML_Analysis/CSV_files/efw_2025-10-31_train.csv",
    "csv_valid": "/mnt/raid/C1_ML_Analysis/CSV_files/efw_2025-10-31_val.csv",
    "csv_test": "/mnt/raid/C1_ML_Analysis/CSV_files/efw_2025-10-31_test.csv",
    "img_column": "file_path",
    "tag_column": "tag",    
    "id_column": "study_id",
    "max_sweeps": 3, 
    "ga_column": None,
    "frame_column": None,
    "presentation_column": None,
    "class_column": None,
    "efw_column": "efw_gt",
    "batch_size": 2,
    "num_frames": 96,
    "num_frames_val": 96, 
    "num_frames_test": 96,
    "num_workers": 16,
    "prefetch_factor": 4,
    "drop_last": False,
    "csv_train_ac": "/mnt/raid/C1_ML_Analysis/CSV_files//c3_blindsweep_annotation_labels_merged_train_train.csv"
}

batch_size = 2
dm = usd.USDataModuleBlindSweepWTag(
        **args_dict
    )
dm.setup()


In [ ]:
train_dl = dm.train_dataloader()
val_dl = dm.val_dataloader()
test_dl = dm.test_dataloader()

In [ ]:
batch = next(iter(train_dl))

# for batch in train_dl:
#     batch

In [ ]:
batch.keys()

In [ ]:
# for batch in val_dl:
#     batch

In [ ]:
df_train = pd.read_csv('/mnt/raid/C1_ML_Analysis/CSV_files/efw_2025-10-31_train.csv')

In [ ]:
print(df_train.query('dataset == "labor"')['study_id'].value_counts().to_string())

In [ ]:
df_train.query('study_id == "IP-025-0335"')['file_path'].values